In [1]:
!pip install -q -U PyPDF2
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 6.4 MB/s eta 0:00:00


In [2]:
FILE_PATH =r'/content/Ai_pyDev.txt'
Resume_pdf="/content/Data Scientist.pdf"

In [3]:
import numpy as np
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import warnings
warnings.filterwarnings('ignore')
import random
import string
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
import pickle
from datetime import date
from IPython.display import Image,display
from IPython.display import Markdown
import textwrap
from docx import Document
from io import BytesIO
from PyPDF2 import PdfReader

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
def open_the_file(path):
    with open (path ,'r',encoding ='utf-8') as f:
        text =f.read()
    return text

 #=======================================================


def get_pdf_text(pdf):
    pdf_reader = PdfReader(BytesIO(pdf))
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    return text

#========================================================


def get_docx_text(docx_bytes):
    document = Document(BytesIO(docx_bytes))
    doc_text = ""
    for para in document.paragraphs:
        doc_text += para.text + "\n"
    return doc_text


#===================================================

def upload_cv(Resume):
    raw_text = ""
    if Resume is not None:
        if Resume.name.endswith(".pdf"):
            raw_text = get_pdf_text(Resume.read())
        elif Resume.name.endswith(".docx"):
            raw_text = get_docx_text(Resume.read())
        return raw_text

#==================================================
def cleaning_text(text):
    cleantext=re.sub('http\S+',' ', text)
    cleantext = re.sub('RT|cc', ' ',cleantext)
    cleantext=re.sub('#\S+\s',' ',cleantext)
    cleantext = re.sub('@\S+', '  ', cleantext)
    cleantext = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleantext)
    cleantext = re.sub(r'[^\x00-\x7f]', ' ', cleantext)
    cleantext = re.sub('\s+', ' ', cleantext)

    return cleantext

 #=======================================================

def Resume(cv):

    def cleaning_text(text):
        cleantext=re.sub('http\S+\s',' ', text)
        cleantext = re.sub('RT|cc', ' ',cleantext)
        cleantext=re.sub('#\S+\s',' ',cleantext)
        cleantext = re.sub('@\S+', '  ', cleantext)
        cleantext = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleantext)
        cleantext = re.sub(r'[^\x00-\x7f]', ' ', cleantext)
        cleantext = re.sub('\s+', ' ', cleantext)

        return cleantext

    with open('/content/tfidf.pkl', 'rb') as f:
        tfidf= pickle.load(f)

    with open('/content/clf.pkl', 'rb') as f:
        clf= pickle.load(f)

    # Map category ID to category name
    category_mapping = {
        15: "Java Developer",
        23: "Testing",
        8: "DevOps Engineer",
        20: "Python Developer",
        24: "Web Designing",
        12: "HR",
        13: "Hadoop",
        3: "Blockchain",
        10: "ETL Developer",
        18: "Operations Manager",
        6: "Data Science",
        22: "Sales",
        16: "Mechanical Engineer",
        1: "Arts",
        7: "Database",
        11: "Electrical Engineering",
        14: "Health and fitness",
        19: "PMO",
        4: "Business Analyst",
        9: "DotNet Developer",
        2: "Automation Testing",
        17: "Network Security Engineer",
        21: "SAP Developer",
        5: "Civil Engineer",
        0: "Advocate",
        }

    for i in range(len(cv)):

        cleaned_resume = cleaning_text(cv)
        # Transform the cleaned resume using the trained TfidfVectorizer
        input_features = tfidf.transform([cleaned_resume])
        # Make the prediction using the loaded classifier
        prediction_id = clf.predict(input_features)[0]

        category_name = category_mapping.get(prediction_id, "Unknown")

        return   category_name

#============================================================

def features_extraction(text):
    try:
        Text = text.lower()

        phone = re.search(r'\d{6,}', Text)  # Modified phone regex to look for 6 or more digits
        Email_1 = re.search(r'https\S+|hppt\S+', Text)  # Updated email regex
        Email_2 = re.search(r'\S+@\S+', Text)  # Another email regex
        Experience = re.findall(r'experience\s+.+', Text)  # Updated experience regex
        Education = re.findall(r'education\s+.+|graduated\s+.+|graduation\s+.+', Text)  # Updated education regex

        data = {'phone': [phone.group() if phone else None],
                'Email_1': [Email_1.group() if Email_1 else None],
                'Email_2': [Email_2.group() if Email_2 else None],
                'Experience': [Experience if Experience else None],
                'Education': [Education if Education else None]}

        d_f = pd.DataFrame(data)

    except Exception as e:
        print(f'Error: {e}')

    return  d_f

#=============================================================

def word_tokenizatian(text):
    word_token = nltk.word_tokenize(str(text))
    return word_token

#================================================


def sent_tokenizatian(text):
    sent_token = nltk.sent_tokenize(str(text))
    return sent_token

#=========================================================

def tokenize_stemmer(text):
    stemmer =SnowballStemmer('english')
    tokens=word_tokenizatian(text.lower())
    stemming=[stemmer.stem(w)for w in tokens]
    return " ".join(stemming)

#========================================================

def response(user_input, file_path=FILE_PATH):
    lemmar = nltk.stem.WordNetLemmatizer()

    def toklemma(tokens):
        return [lemmar.lemmatize(tk) for tk in tokens]

    remove_puc_dict = dict((ord(puc), None) for puc in string.punctuation)

    def lemNormalize(text):
        return toklemma(nltk.word_tokenize(text.lower().translate(remove_puc_dict)))

    with open(file_path, 'r',encoding ='utf-8') as file:
        data = file.read()

    def sent_tokenization(text):
        sent_token = nltk.sent_tokenize(str(text))
        return sent_token

    Sent_token = sent_tokenization(data)

    # Append user input to the tokenized sentences
    Sent_token.append(user_input)

    tfidfvec = TfidfVectorizer(tokenizer=lemNormalize, stop_words='english')
    tfidf = tfidfvec.fit_transform(Sent_token)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    reg_tfidf = flat[-2]

    if reg_tfidf == 0:
        robo1_res = "I am sorry, I don't understand you"
        return robo1_res
    else:
        robo1_res = Sent_token[idx]
        return robo1_res


#==================================================================

def similarity(text_1,text_2):
    TFIDF=TfidfVectorizer(tokenizer=tokenize_stemmer,stop_words='english')
    tfidfmatrix =TFIDF.fit_transform([text_1,text_2])
    similar_vector = cosine_similarity(tfidfmatrix)[0][1]
    return similar_vector

#=======================================================================

#Create a helper function that will convert the markdown into nicely formatted text
def to_markdown(text):
    text = text.replace('•','*')
    return Markdown(textwrap.indent(text, '>', predicate=lambda _: True))

#====================================================================

In [5]:
data =open_the_file(FILE_PATH)
to_markdown(data[:100])

>Explain Artificial Intelligence and give its applications.,Artificial Intelligence (AI) is a field o

In [6]:
with open('Data Scientist.pdf', 'rb') as Resume_pdf:
    CV = upload_cv(Resume_pdf)

In [7]:
with open('Data Scientist.docx', 'rb') as Resume_pdf:
    Resume_docx = upload_cv(Resume_pdf)

In [8]:
to_markdown(Resume(CV))

>Data Science

In [9]:
features_extraction(Resume_docx)

,phone,Email_1,Email_2,Experience,Education
0,2001149973327,None,mahmoudibrahimabdelfattah48@gmail.com,[experience\n\n04/2022 – 08/2024\tdata scienti...,[education\nbachelor's degree in computer scie...


In [10]:
word_token =word_tokenizatian(data)
len(word_token)

13669

In [11]:
sent_token=sent_tokenizatian(data)
len(sent_token)

548

In [12]:
to_markdown(cleaning_text(response('what is the machine learning')))

>Machine Learning Machine learning is essential for cognitive computing as it allows systems to learn from data adapt and improve their performance over time 

In [13]:
to_markdown(cleaning_text(response('what built-in types are available in Python?')))

>What built in types are available in Python The built in types in Python include Strings Integer Complex numbers Floating point numbers Built in functions How do you differentiate between py and pc files in Python A file in Python with extension py are source files while with extension pyc are compiled bytecode files generated by the compiler 

In [14]:
similarity('the book is very good','the cat is vary cute')

0.5133943255091278

In [16]:
greet_input = ['hey','hello','greetings','sup','whats up','hi']

greet_response = ['hey dear','helo dear ','hi there','whats going one','I am glad! you are talking to me']

In [17]:
Score =[]
Data =[]
Date=[]
users=[]

In [18]:
def Extract_informatian(DATA):
    columns =['phone','Email_1','Email_2','Experience','Education']
    df =pd.DataFrame(columns =columns)
    data =pd.concat([DATA,df],axis =0)
    Data.append(data)

In [19]:
def interactive(X1,X2,X3,X4):
    columns =['question_model','Answer_user','question_user','Answer_model']
    df =pd.DataFrame(columns =columns)
    data =pd.concat([X1,df],axis =0)
    users.append(data)

In [20]:
def ChatBot(data):

    flag =True

    print('''Bot: hey me name is ZOOX,i am an artificial intelligence program to help you get the job,
           Let's have a conversation. If you want to exit just write bye!''')
    print()
    #display(Image(filename =image_path, width =200, height =400))

    while flag==True:

        print()
        today =date.today()
        print('interview date is :',today)
        Date.append(today)
        print()
        print()

        Features =features_extraction(data)
        print(Features)
        Extract_informatian(Features)

        predict =Resume(data)
        print()
        print( 'job Expected  =' , predict)

        if predict =='Data Science':


            questian1 =str(input('How are machine learning and AI related?'))
            questian1 =questian1.lower()
            similarity1 =similarity(questian1,response('How are machine learning and AI related?'))
            print("The Similarity of your Answer and correct answer dear = ",similarity1)

            questian2 =str(input('We use AI to build various applications for example ?'))
            questian2 = questian2.lower()
            similarity2 =similarity(questian2,response('We use AI to build various applications for example ?'))

            print("The Similarity of your Answer and correct answer dear = ",    similarity2)

            questian3 =str(input('Explain Artificial Intelligence and give its applications?'))
            questian3 =questian3.lower()
            similarity3 =similarity(questian3,response('Explain Artificial Intelligence and give its applications?'))

            print("The Similarity of your Answer and correct answer dear = ",similarity3)

            questian4 =str(input('Give some examples of weak and strong AI?'))
            questian4 = questian4.lower()
            similarity4 = similarity(questian4,response('Give some examples of weak and strong AI?'))
            print("The Similarity of your Answer and correct answer dear  = ", similarity4)
            print('----------')
            total =(similarity1+similarity2+similarity3+similarity4)/4*100
            print("The Score of The interview =",total)
            Score.append(total)
            flag =False


        elif predict =='Python Developer':

            questian1 =str(input('What built-in types are available in Python?'))
            questian1 =questian1.lower()
            similarity1 =similarity(questian1,response('What built-in types are available in Python?'))
            print("The Similarity of your Answer and correct answer dear = ",similarity1)
            Score.append(similarity1)
            flag =False
            break

    for i in range(0,1000):
        user_response=input('you')
        if user_response in greet_input:
            print('Bot :',random.choice(greet_response))

        elif user_response !='bye':
            if user_response=="thanks" or user_response=='thank you':
                flag=False
                print("Bot: You are Welcome")
                break
            else:
                sent_token.append(user_response)
                word_tokens =word_token+word_tokenizatian(user_response)
                final_words = list(set(word_tokens))

                print("Bot:",end="")
                print('-'*5)
                print(response(user_response))
                sent_token.remove(user_response)

        else:

            flag=False
            print('Bot: Ok Good Bye! Take Care')
            break


##pdf هنا احنا بنقرا السيره الذاتيه الصيغه

In [21]:
with open('Data Scientist.pdf', 'rb') as Resume_pdf:
    CV = upload_cv(Resume_pdf)

##هنا احنا بنقرا السيره الذاتيه الصيغه الورد


In [22]:
with open('Data Scientist.docx', 'rb') as Resume_pdf:
    resume_docx = upload_cv(Resume_pdf)

### big Data
### text classificatian
#### deep learning
####  machine learning is weak and deep learning is stronge

In [23]:
ChatBot(resume_docx)

Bot: hey me name is ZOOX,i am an artificial intelligence program to help you get the job,
           Let's have a conversation. If you want to exit just write bye!


interview date is : 2024-07-07


           phone Email_1                                Email_2  \
0  2001149973327    None  mahmoudibrahimabdelfattah48@gmail.com   

                                          Experience  \
0  [experience\n\n04/2022 – 08/2024\tdata scienti...   

                                           Education  
0  [education\nbachelor's degree in computer scie...  

job Expected  = Data Science
How are machine learning and AI related?big Data
The Similarity of your Answer and correct answer dear =  0.46798240551243636
We use AI to build various applications for example ?text classificatian
The Similarity of your Answer and correct answer dear =  0.4527404405779374
Explain Artificial Intelligence and give its applications?deep learning
The Similarity of your Answer and correct answer dear =  0.5131843

In [24]:
def details_interview(Score,Date,Data):
    """
    the functian to make combination All Data

    """
    score =np.array(Score)
    score=pd.Series(score,name='Score')
    score =pd.DataFrame(score)
    score['interview date'] =pd.to_datetime(Date)
    columns =['phone','Email_1','Email_2','Experience','Education']
    data=np.reshape(Data,(-1,5))
    data =pd.DataFrame(data,columns=columns)
    DATA=pd.concat([score,data],ignore_index =False,axis=1)
    return DATA

In [25]:
data =details_interview(Score,Date,Data)
data.head()

,Score,interview date,phone,Email_1,Email_2,Experience,Education
0,58.572859,2024-07-07,2001149973327,None,mahmoudibrahimabdelfattah48@gmail.com,[experience\n\n04/2022 – 08/2024\tdata scienti...,[education\nbachelor's degree in computer scie...


In [26]:
data.to_excel('result_HR.xlsx')